## Centrality Dependence of CNM Effects

In [1]:
# CUDA 12.x example
# !pip install -U cupy-cuda12x
import cupy as cp; x=cp.arange(5); print(x.device)

<CUDA Device 0>


In [2]:
# npdf_gpu_pipeline.py
# Fast, GPU-aware (CuPy) RpA driver for RAW + BINNED + HESSIAN bands.
from __future__ import annotations
import numpy as np
try:
    import cupy as cp
    XP = cp                         # GPU
    def xp_asnumpy(a): return cp.asnumpy(a)
    def xp_array(a, **kw): return cp.asarray(a, **kw)
    GPU = True
except Exception:
    XP = np                         # CPU fallback
    def xp_asnumpy(a): return np.asarray(a)
    def xp_array(a, **kw): return np.asarray(a, **kw)
    GPU = False

from npdf_data   import NPDFSystem, RpAAnalysis
from gluon_ratio import EPPS21Ratio, GluonEPPSProvider
from glauber     import OpticalGlauber, SystemSpec

# --------- small utils ---------
def _pairwise_hessian_band(vals49: np.ndarray, axis=0, cl68=False):
    """
    vals49: shape (..., 49, ...), index 0 = central, 1..48 = error members in EPPS pair order
    Returns (central, lo, hi) along 'axis', using symmetric Hessian (90% CL).
    """
    # move 'axis' to the front for convenience
    v = np.moveaxis(vals49, axis, 0)   # [49, ...]
    c = v[0]
    diffsq = 0.0
    # pairs: (2,3), (4,5), ... (48,49)  -> 1-based -> 1..48 zero-based: (1,2),(3,4),...
    for k in range(1, 49, 2):
        d = v[k] - v[k+1]
        diffsq = diffsq + d*d
    delta90 = 0.5 * np.sqrt(diffsq)
    delta = delta90 / 1.645 if cl68 else delta90
    return c, c - delta, c + delta

def _weighted_avg_perbin(values, weights, bin_idx, nbins):
    """
    values: (..., N)  (can be 49 sets x N or 1 x N)
    weights: (N,)
    bin_idx: (N,) int64 in [0..nbins-1] or -1 for out of range
    returns (..., nbins)
    """
    # reshape to [M, N]
    M = int(values.shape[0])
    N = int(values.shape[-1])
    vals = values.reshape(M, N)

    # mask out-of-range
    valid = bin_idx >= 0
    if GPU:
        valid_idx = XP.where(valid)[0]
    else:
        valid_idx = np.where(valid)[0]
    if valid_idx.size == 0:
        return XP.zeros((M, nbins), dtype=values.dtype)

    idx = bin_idx[valid_idx]
    w   = weights[valid_idx]
    v   = vals[:, valid_idx]            # [M, Nv]

    # numerator/denominator using bincount; loop M (49) is cheap
    out = XP.zeros((M, nbins), dtype=values.dtype)
    den = XP.bincount(idx, weights=w, minlength=nbins)
    den = XP.maximum(den, XP.asarray(1e-30, den.dtype))
    for m in range(M):
        num = XP.bincount(idx, weights=v[m]*w, minlength=nbins)
        out[m] = num / den
    return out

def _bin_edges(start, stop, step):
    # robust edges inclusive of 'stop'
    n = int(np.floor((stop - start) / step + 1e-12)) + 1
    return np.linspace(start, start + n*step, n+1)

# --------- core loaders (CPU) ---------
def load_R0_and_weights(top_folder: str):
    """
    Returns:
      base_df           : DataFrame with (y, pt)
      R0_sets_49xN      : np.ndarray float64
      sigma_pA_central  : np.ndarray float64 weights aligned to base_df
    """
    ana = RpAAnalysis()
    sys = NPDFSystem.from_folder(top_folder, kick="pp")
    base, r0_central, M = ana.compute_rpa_members(sys.df_pp, sys.df_pa, sys.df_errors, join="intersect")
    R0 = np.vstack([r0_central[None, :], M])   # [49, N]
    # weights = σ_pA^central aligned to base grid
    wtab = ana._make_weight_table(base.assign(r_central=r0_central), sys.df_pa, df_pp=sys.df_pp, mode="pa")
    w = wtab["w"].to_numpy(float)              # [N]
    return base, R0.astype(np.float64), w.astype(np.float64)

def precompute_SA_sets(provider: GluonEPPSProvider, yv: np.ndarray, pv: np.ndarray):
    """
    Compute and cache gluon S_A for all 49 sets on the (y,pt) grid.
    Returns SA_sets: [49, N]
    """
    SA = np.empty((49, yv.size), dtype=np.float64)
    for sid in range(1, 50):    # 1..49
        SA[sid-1] = np.asarray(provider.SA_ypt_set(yv, pv, set_id=sid, flav="g"), float)
    return SA

def K_sets_from_alpha(SA_49xN: np.ndarray, alpha_bar: float, Nnorm: float):
    """
    K = (1 + Nnorm*(S_A-1)*alpha_bar) / S_A  → shape [49, N]
    """
    SA = SA_49xN
    num = 1.0 + Nnorm * (SA - 1.0) * float(alpha_bar)
    den = np.clip(SA, 1e-15, None)
    return num / den

# --------- high-level reports ---------
class NPDFGPUDriver:
    def __init__(self, *,
                 pPb_dir: str,
                 epps_dir: str,
                 sqrt_sNN_GeV: float,
                 sigmaNN_mb: float = 71.0,
                 A: int = 208,
                 m_state: str | float = "charmonium",
                 y_sign_for_xA: int = -1):
        # 1) R0 & weights (CPU)
        self.base, self.R0_49xN, self.wN = load_R0_and_weights(pPb_dir)
        self.y = self.base["y"].to_numpy(float)
        self.pt = self.base["pt"].to_numpy(float)
        # 2) EPPS gluon provider + geometry (CPU)
        self.provider = GluonEPPSProvider(
            EPPS21Ratio(A=A, path=epps_dir),
            sqrt_sNN_GeV=float(sqrt_sNN_GeV),
            m_state_GeV=m_state,
            y_sign_for_xA=y_sign_for_xA
        ).with_geometry(None)
        self.geom = self.provider._geom
        self.Nnorm = float(self.geom.Nnorm())
        self.sigmaNN_mb = float(sigmaNN_mb)

        # 3) Precompute S_A for all sets on the grid (CPU, cached)
        self.SA_49xN = precompute_SA_sets(self.provider, self.y, self.pt)

        # 4) Move big arrays to GPU (if available)
        self.R0_x = xp_array(self.R0_49xN)
        self.SA_x = xp_array(self.SA_49xN)
        self.w_x  = xp_array(self.wN)

    # ---------- MIN-BIAS (K=1) ----------
    def rpa_mb_raw(self):
        """RAW min-bias: returns R0 for all sets on the unbinned (y,pt) points."""
        return self.R0_x  # [49, N], central in row 0

    # ---------- CENTRALITY ----------
    def rpa_vs_centrality(self, cent_edges=(0,20,40,60,80,100),
                          y_rng=(-5,5), pt_rng=(0,20)):
        """
        Weighted average over chosen (y,pt) window, reported per centrality bin.
        Returns dict with keys 'cent_left', 'central', 'lo', 'hi'.
        """
        y0, y1 = float(y_rng[0]), float(y_rng[1])
        p0, p1 = float(pt_rng[0]), float(pt_rng[1])
        sel = (self.y >= y0) & (self.y < y1) & (self.pt >= p0) & (self.pt < p1)
        if not np.any(sel):
            return dict(cent_left=np.asarray([]), central=np.asarray([]), lo=np.asarray([]), hi=np.asarray([]))

        # slice grid
        R0 = self.R0_x[:, sel]      # [49, Ns]
        SA = self.SA_x[:, sel]      # [49, Ns]
        w  = self.w_x[sel]          # [Ns]

        # build alpha_bar per bin from Glauber (CPU; scalar per bin)
        cent_edges = list(cent_edges)
        out_vals = []
        for i in range(len(cent_edges)-1):
            cL, cR = cent_edges[i], cent_edges[i+1]
            a_bar = float(self.geom.alpha_bar_for_bin(cL, cR, sigmaNN_mb=self.sigmaNN_mb))
            K = xp_array(K_sets_from_alpha(xp_asnumpy(SA), a_bar, self.Nnorm))   # compute on CPU, send to GPU
            R = R0 * K  # [49, Ns]
            # weighted average over the slice
            num = XP.sum(R * w, axis=1)   # [49]
            den = XP.sum(w) + XP.asarray(1e-30)
            mean_per_set = num / den      # [49]
            out_vals.append(mean_per_set)

        # shape → [nbins, 49] → numpy
        V = xp_asnumpy(XP.stack(out_vals, axis=0))  # [B, 49]
        # Hessian per bin
        central, lo, hi = [], [], []
        for b in range(V.shape[0]):
            c, l, h = _pairwise_hessian_band(V[b, :][None, ...], axis=1)  # feed [1,49]
            central.append(float(c.ravel()[0])); lo.append(float(l.ravel()[0])); hi.append(float(h.ravel()[0]))
        return dict(
            cent_left=np.asarray(cent_edges[:-1], float),
            central=np.asarray(central, float),
            lo=np.asarray(lo, float),
            hi=np.asarray(hi, float),
        )

    # ---------- BINNED vs y (min-bias, then bands) ----------
    def rpa_vs_y(self, y_edges=np.arange(-5.0, 5.0+1e-9, 0.5), pt_rng=(0,20)):
        p0, p1 = float(pt_rng[0]), float(pt_rng[1])
        sel = (self.pt >= p0) & (self.pt < p1)
        R0 = self.R0_x[:, sel]     # [49, Ns]
        w  = self.w_x[sel]
        yy = XP.asarray(self.y[sel])
        # bin index
        be = XP.asarray(y_edges)
        # digitize half-open [left,right)
        idx = XP.digitize(yy, be) - 1
        # set out-of-range to -1
        idx = XP.where((idx >= 0) & (idx < (len(y_edges)-1)), idx, -1)
        # per-bin weighted means per set
        means = _weighted_avg_perbin(R0, w, idx, nbins=len(y_edges)-1)  # [49, Nybins]
        V = xp_asnumpy(means)   # numpy
        # Hessian per bin
        C, L, H = _pairwise_hessian_band(V, axis=0)  # bins along axis 1 → pass axis=0 after our shape
        return dict(y_left=np.asarray(y_edges[:-1], float), central=C, lo=L, hi=H)

    # ---------- BINNED vs pT (min-bias, then bands) ----------
    def rpa_vs_pt(self, pt_edges=np.arange(0.0, 20.0+1e-9, 2.5), y_rng=(-5,5)):
        y0, y1 = float(y_rng[0]), float(y_rng[1])
        sel = (self.y >= y0) & (self.y < y1)
        R0 = self.R0_x[:, sel]     # [49, Ns]
        w  = self.w_x[sel]
        pp = XP.asarray(self.pt[sel])
        be = XP.asarray(pt_edges)
        idx = XP.digitize(pp, be) - 1
        idx = XP.where((idx >= 0) & (idx < (len(pt_edges)-1)), idx, -1)
        means = _weighted_avg_perbin(R0, w, idx, nbins=len(pt_edges)-1)  # [49, Npbins]
        V = xp_asnumpy(means)
        C, L, H = _pairwise_hessian_band(V, axis=0)
        return dict(pt_left=np.asarray(pt_edges[:-1], float), central=C, lo=L, hi=H)

ModuleNotFoundError: No module named 'npdf_data'

In [ ]:
# --- inputs (edit to your paths/energies) ---
P5   = "./input/npdf/pPb5TeV"
P8   = "./input/npdf/pPb8TeV"
EPPS = "./input/npdf/nPDFs"

# 5.02 TeV
drv5 = NPDFGPUDriver(pPb_dir=P5, epps_dir=EPPS, sqrt_sNN_GeV=5023.0, sigmaNN_mb=67.2, A=208)

# 8.16 TeV
drv8 = NPDFGPUDriver(pPb_dir=P8, epps_dir=EPPS, sqrt_sNN_GeV=8160.0, sigmaNN_mb=71.0,  A=208)

CENT_EDGES = (0,20,40,60,80,100)
Y_EDGES    = np.arange(-5.0, 5.0+1e-9, 0.5)
PT_EDGES   = np.arange( 0.0,20.0+1e-9, 2.5)

In [ ]:
# 1) RpA vs centrality (choose a rapidity/pT window, e.g. midrapidity, all pT)
r_cent_5 = drv5.rpa_vs_centrality(CENT_EDGES, y_rng=(-1.37,0.43), pt_rng=(0,20))
r_cent_8 = drv8.rpa_vs_centrality(CENT_EDGES, y_rng=(-1.37,0.43), pt_rng=(0,20))

In [ ]:
# 2) RpA vs y (min-bias) in 0–20 GeV
r_y_5 = drv5.rpa_vs_y(Y_EDGES, pt_rng=(0,20))
r_y_8 = drv8.rpa_vs_y(Y_EDGES, pt_rng=(0,20))

In [ ]:
# 3) RpA vs pT (min-bias) in three y-windows (example)
mid = drv5.rpa_vs_pt(PT_EDGES, y_rng=(-1.37,0.43))
fwd = drv5.rpa_vs_pt(PT_EDGES, y_rng=( 2.03,3.53))
bwd = drv5.rpa_vs_pt(PT_EDGES, y_rng=(-4.46,-2.96))

In [ ]:
# 4) RAW (no binning): per (y,pt) grid, all 49 sets (min-bias baseline)
R0_raw_5 = drv5.rpa_mb_raw()  # shape [49, N] aligned to drv5.base[['y','pt']]

## Step 2

In [ ]:
# step2_rpa_vs_centrality.py
from __future__ import annotations
import numpy as np
try:
    import cupy as cp
    xp = cp
    GPU = True
except Exception:
    xp = np
    GPU = False

from dataclasses import dataclass
from typing import Tuple, List, Dict, Sequence
from pathlib import Path

In [ ]:
# ---- small helpers ----
def _to_xp(a):
    return xp.asarray(a) if not isinstance(a, xp.ndarray) else a

def _to_np(a):
    return cp.asnumpy(a) if GPU and isinstance(a, cp.ndarray) else np.asarray(a)

def _bin_widths(b: np.ndarray) -> np.ndarray:
    b = np.asarray(b, float)
    if b.size == 1:
        return np.array([1.0], float)
    dx = np.empty_like(b)
    dx[1:-1] = 0.5*(b[2:] - b[:-2])
    dx[0]    = 0.5*(b[1] - b[0])
    dx[-1]   = 0.5*(b[-1] - b[-2])
    return dx

def _hessian_band(values49: xp.ndarray) -> Tuple[xp.ndarray, xp.ndarray, xp.ndarray]:
    """
    values49: shape (49, ...) with [0]=central, [1..48]=Hessian members in (−,+) pairs
    Returns (central, lo, hi) with symmetric Hessian at 68%CL (EPPS21 default is 90% — rescale if you want).
    """
    C = values49[0]
    # members are assumed to be ordered as (2,3), (4,5), ..., (48,49) in 1-based EPPS → here (1,2), (3,4), ...
    diffsq = xp.zeros_like(C)
    for k in range(1, 49, 2):
        dm = values49[k]   # "minus"
        dp = values49[k+1] # "plus"
        diffsq = diffsq + (dp - dm)**2
    d90 = 0.5 * xp.sqrt(diffsq)
    d68 = d90 / 1.645   # convert 90%→68% if you prefer 68% CL
    return C, C - d68, C + d68

In [ ]:
@dataclass
class CentralityBin:
    tag: str
    bL: float
    bR: float
    w_b: np.ndarray     # weights on your raw b-grid, normalized inside [bL,bR]

def centrality_bins_from_glauber(glb, b_grid: np.ndarray, edges_pct: Sequence[float]) -> List[CentralityBin]:
    """
    Map centrality percent edges → (b_L, b_R), then build inelastic weights over your discrete b-grid.
    """
    # 1) get CDF arrays from Glauber
    cum = glb.cum_pA           # cumulative P_inel(b) normalized to 1
    bG  = glb.b_grid           # fine Glauber b grid (0..~20 fm)
    # 2) inverse CDF for each edge
    def b_at_percent(pct):
        c = float(pct)/100.0
        return float(np.interp(c, _to_np(cum), _to_np(bG)))
    # 3) inelastic weight density ~ 2π b (1 - exp(-σ T(b))) — use Glauber’s arrays
    #    We approximate on your discrete b_grid by interpolating TpA(b) to those points.
    sigma_fm2 = glb.sigma_nn_mb * 0.1
    TpA_at_raw = np.interp(b_grid, _to_np(glb.b_grid), _to_np(glb.TpA_b))
    Pinel = 1.0 - np.exp(-sigma_fm2 * np.maximum(TpA_at_raw, 0.0))
    db = _bin_widths(b_grid)
    base_w = 2.0*np.pi*b_grid*db*Pinel
    bins: List[CentralityBin] = []
    for i in range(len(edges_pct)-1):
        L, R = b_at_percent(edges_pct[i]), b_at_percent(edges_pct[i+1])
        m = (b_grid >= L) & (b_grid < R)
        w = base_w * m
        s = w.sum()
        w = w/s if s > 0 else w
        bins.append(CentralityBin(f"{edges_pct[i]}-{edges_pct[i+1]}%", L, R, w))
    return bins

def average_over_b_sets(R_49byp: xp.ndarray, w_b: np.ndarray) -> xp.ndarray:
    """
    R_49byp: (49, Nb, Ny, Np)
    w_b:     (Nb,) numpy weights (will be moved to xp)
    returns: (49, Ny, Np)  weighted average over b
    """
    W = _to_xp(w_b.reshape(1, -1, 1, 1))
    num = xp.sum(R_49byp * W, axis=1)
    # W sums to 1 within the bin by construction → denominator=1
    return num

def average_over_ypt_sets(R_49yp: xp.ndarray,
                          y_grid: np.ndarray, pt_grid: np.ndarray,
                          y_win: Tuple[float,float],
                          pt_win: Tuple[float,float],
                          sigma_pa_central: np.ndarray) -> xp.ndarray:
    """
    R_49yp: (49, Ny, Np)
    sigma_pa_central: (Ny, Np) weights (pp is acceptable too; we use σ_pA central by default)
    returns: (49,) weighted average over (y, pT)
    """
    y = np.asarray(y_grid); p = np.asarray(pt_grid)
    mY = (y >= y_win[0]) & (y <= y_win[1])
    mP = (p >= pt_win[0]) & (p <= pt_win[1])
    if not mY.any() or not mP.any():
        return xp.full((49,), xp.nan)

    # slice
    R = R_49yp[:, mY][:, :, mP]               # (49, nY, nP)
    W = _to_xp(np.clip(sigma_pa_central[mY][:, mP], 0, None))
    s = xp.sum(W)
    if s <= 0:
        return xp.full((49,), xp.nan)
    return xp.sum(R * W, axis=(1,2)) / s

def rpa_vs_centrality_with_band(raw: Dict,
                                glb,                        # OpticalGlauber instance (matching energy)
                                sigma_pa_central: np.ndarray,  # (Ny, Np) from your npdf_data sys.df_pa central
                                edges_pct=(0,20,40,60,80,100),
                                y_win=(-5,5), pt_win=(0,20)) -> Dict:
    """
    Compute: for each centrality bin → 49-set RpA averaged over b (bin) and then over (y,pT),
    plus a min-bias (0–100%) horizontal band.
    Returns dict with arrays ready to plot: centers (%), (r_c, r_lo, r_hi), and MB band.
    """
    # unpack raw
    b_grid = np.asarray(raw["b"], float)
    y_grid = np.asarray(raw["y"], float)
    p_grid = np.asarray(raw["pt"], float)
    R      = _to_xp(raw["R"])                 # (49, Nb, Ny, Np)

    # centrality bins (weights on your b_grid)
    cbins = centrality_bins_from_glauber(glb, b_grid, edges_pct)

    # per-bin 49-set averages
    r49_per_bin = []
    for C in cbins:
        R_49yp = average_over_b_sets(R, C.w_b)                # (49, Ny, Np)
        r49 = average_over_ypt_sets(R_49yp, y_grid, p_grid, y_win, pt_win, sigma_pa_central)
        r49_per_bin.append(_to_np(r49))                       # (49,)

    r49_per_bin = np.stack(r49_per_bin, axis=0)               # (Nbin, 49)
    # build Hessian band per bin
    rc, rlo, rhi = [], [], []
    for i in range(r49_per_bin.shape[0]):
        C, L, H = _hessian_band(_to_xp(r49_per_bin[i]))
        rc.append(_to_np(C)); rlo.append(_to_np(L)); rhi.append(_to_np(H))
    rc, rlo, rhi = map(np.asarray, (rc, rlo, rhi))            # (Nbin,)

    # min-bias (0–100%) once
    Cmb = centrality_bins_from_glauber(glb, b_grid, (0,100))[0]
    Rmb_49yp = average_over_b_sets(R, Cmb.w_b)                # (49, Ny, Np)
    rmb_49   = _to_np(average_over_ypt_sets(Rmb_49yp, y_grid, p_grid, y_win, pt_win, sigma_pa_central))
    Cmb_c, Cmb_lo, Cmb_hi = map(_to_np, _hessian_band(_to_xp(rmb_49)))

    # x (bin centers in %)
    cent_edges = np.asarray(edges_pct, float)
    cent_mid   = 0.5*(cent_edges[:-1] + cent_edges[1:])

    return dict(
        cent_mid=cent_mid, cent_edges=cent_edges,
        r_c=rc, r_lo=rlo, r_hi=rhi,
        mb_c=float(Cmb_c), mb_lo=float(Cmb_lo), mb_hi=float(Cmb_hi)
    )

# --------- tiny plot helper (matplotlib) ----------
def plot_vs_centrality(res_5, res_8, title="", note="", outfile=None):
    import matplotlib.pyplot as plt
    def _stairs(x_left, y):
        x = np.asarray(x_left, float)
        dx = np.diff(x); dx_last = dx[-1] if dx.size else 20.0
        return np.r_[x, x[-1]+dx_last], np.r_[y, y[-1]]

    fig, ax = plt.subplots(1, 1, figsize=(6.8, 4.2), constrained_layout=True)
    for res, color, lab in [(res_5, "tab:blue", "5.02 TeV"), (res_8, "tab:red", "8.16 TeV")]:
        xL = res["cent_edges"][:-1]
        for y, lo, hi, label in [(res["r_c"], res["r_lo"], res["r_hi"], fr"{lab}")]:
            X, Yc = _stairs(xL, y); _, Ylo = _stairs(xL, lo); _, Yhi = _stairs(xL, hi)
            ax.step(X, Yc, where="post", lw=2, color=color, label=label)
            ax.fill_between(X, Ylo, Yhi, step="post", alpha=0.25, color=color)
        # min-bias band (horizontal)
        ax.axhspan(res["mb_lo"], res["mb_hi"], alpha=0.18, color=color, lw=0)
        ax.axhline(res["mb_c"], color=color, ls="--", lw=1.4, alpha=0.9)

    ax.set_xlabel("Centrality [%]"); ax.set_ylabel(r"$R_{pA}$")
    ax.set_xlim(0, 100); ax.set_ylim(0.35, 1.25)
    if note: ax.text(0.02, 0.02, note, transform=ax.transAxes, ha="left", va="bottom",
                     bbox=dict(facecolor="white", alpha=0.85, edgecolor="none"))
    ax.legend(frameon=False)
    if title: ax.set_title(title)
    if outfile:
        Path(outfile).parent.mkdir(parents=True, exist_ok=True)
        fig.savefig(outfile, bbox_inches="tight")
    plt.show()

In [ ]:
# In your notebook/session right after Step 1 (where raw5/raw8 are built):
from glauber import OpticalGlauber, SystemSpec
from npdf_data import NPDFSystem   

# --- Glauber per energy (σ_NN mb that you already use) ---
glb5 = OpticalGlauber(SystemSpec(system="pA", roots_GeV=5023.0, A=208, sigma_nn_mb=67.2), verbose=False)
glb8 = OpticalGlauber(SystemSpec(system="pA", roots_GeV=8160.0, A=208, sigma_nn_mb=71.0),  verbose=False)

# --- σ_pA central weights on the SAME (y,pt) grid you used to make raw tables ---
sys5 = NPDFSystem.from_folder("./input/npdf/pPb5TeV", kick="pp")
sys8 = NPDFSystem.from_folder("./input/npdf/pPb8TeV", kick="pp")

# Build a σ_pA(y,pt) central weight grid aligned to raw's (y,pt):
def sigma_pa_central_on_raw_grid(sysX, rawX):
    # nearest-neighbor remap is fine because your raw grid was built from these same slabs
    # make a tiny map from (y,pt)->σ
    from collections import defaultdict
    tab = defaultdict(dict)
    for y,pt,val in sysX.df_pa[["y","pt","val"]].to_numpy():
        tab[float(y)][float(pt)] = float(val)
    yv, pv = rawX["y"], rawX["pt"]
    Y, P = np.meshgrid(yv, pv, indexing="ij")
    W = np.empty_like(Y, float)
    for i, yy in enumerate(yv):
        row = tab.get(float(yy), {})
        # fallback: nearest pt by min |Δpt|
        pts_row = np.array(list(row.keys())) if row else np.array([pv[0]])
        for j, pp in enumerate(pv):
            if pp in row:
                W[i,j] = row[pp]
            else:
                jn = int(np.argmin((pts_row-pp)**2))
                W[i,j] = row.get(pts_row[jn], 0.0)
    return W

In [ ]:
W5 = sigma_pa_central_on_raw_grid(sys5, raw5)  # shape (Ny, Np)
W8 = sigma_pa_central_on_raw_grid(sys8, raw8)

# --- choose windows & bins ---
CENT_EDGES = (0,20,40,60,80,100)
Y_WINDOWS  = [(-4.46,-2.96), (-1.37,0.43), (2.03,3.53)]
PT_RANGE   = (0.0, 20.0)  # change to (2.5,20) etc.

# --- make the 3-window figure (call once per window or loop) ---
for y_win, tag in zip(Y_WINDOWS, ["Backward","Mid","Forward"]):
    res5 = rpa_vs_centrality_with_band(raw5, glb5, W5, CENT_EDGES, y_win=y_win, pt_win=PT_RANGE)
    res8 = rpa_vs_centrality_with_band(raw8, glb8, W8, CENT_EDGES, y_win=y_win, pt_win=PT_RANGE)
    plot_vs_centrality(
        res5, res8,
        title=f"RpA vs Centrality — {tag} rapidity",
        note=fr"{y_win[0]} < y < {y_win[1]},  {PT_RANGE[0]} ≤ $p_T$ ≤ {PT_RANGE[1]} GeV",
        outfile=f"./output-npdf/step2_rpa_vs_centrality_{tag}.pdf"
    )